# FEATURE ENGINEERING 
- Create dataframe series which specifies if location and keywords are missing
- Create a model with this columns (depending on the metrics generated)
- Create a pandas series which specifies if the text has an hashtag
- Combine with text model

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


## CREATE DATAFRAME SERIES (empty_location and empty_keyword)

In [16]:
train["keyword"] = train["keyword"].fillna("None")
train["location"] = train["location"].fillna("None")

In [17]:
def empty_keyword(df):
    if df == "None":
        return 0
    else:
        return 1

In [18]:
train["empty_keyword"] = train["keyword"].apply(empty_keyword)
train["empty_location"] = train["location"].apply(empty_keyword)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
id                7613 non-null int64
keyword           7613 non-null object
location          7613 non-null object
text              7613 non-null object
target            7613 non-null int64
empty_keyword     7613 non-null int64
empty_location    7613 non-null int64
dtypes: int64(4), object(3)
memory usage: 416.5+ KB


In [59]:
train["keyword"].value_counts()

None                     61
fatalities               45
armageddon               42
deluge                   42
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 222, dtype: int64

In [20]:
train["empty_keyword"].value_counts()

1    7552
0      61
Name: empty_keyword, dtype: int64

In [21]:
train["empty_location"].value_counts()

1    5080
0    2533
Name: empty_location, dtype: int64

In [22]:
train["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [23]:
feature_cols = ["empty_keyword", "empty_location"]

In [24]:
X = train[feature_cols]
X.head()

,empty_keyword,empty_location
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [54]:
X_eins = train["empty_location"]
X_eins = X_eins.values.reshape(-1, 1)

In [25]:
y = train["target"]

## CREATE A MODEL WITH empty_location and empty_keyword

In [52]:
from sklearn.model_selection import cross_val_score

def run_model(model):
    return cross_val_score(model, X, y, cv = 5, scoring = "accuracy").mean()

def run_model_eins(model):
    return cross_val_score(model, X_eins, y, cv = 5, scoring = "accuracy").mean()

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

lr = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [55]:
print(run_model(lr))
print(run_model_eins(lr))

0.5702089760861372
0.5703402096819377


In [56]:
print(run_model(nb))
print(run_model_eins(nb))

0.5703402096819377
0.5703402096819377


In [57]:
print(run_model(rf))
print(run_model_eins(rf))

0.5702089760861372
0.5703402096819377


In [58]:
print(run_model(xgb))
print(run_model_eins(xgb))

0.5702089760861372
0.5703402096819377


#### Conclusion: the scores are not much different from the NULL-MODEL 

## PANDAS SERIES INDICATING PRESENCE OF HASHTAGS

In [3]:
train["text"].head()

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [26]:
import re

def hashtag(df):
    if re.search(r'#\w+', df):
        return 1
    else:
        return 0
    
    
def empty_keyword(df):
    if df == "None":
        return 0
    else:
        return 1

def run_model(model):
    return cross_val_score(model, X, y, cv = 5, scoring = "accuracy").mean()

In [22]:
train["hashtag"] = train["text"].apply(hashtag)
train["empty_location"] = train["location"].apply(empty_keyword)

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
id                7613 non-null int64
keyword           7552 non-null object
location          5080 non-null object
text              7613 non-null object
target            7613 non-null int64
hashtag           7613 non-null int64
empty_location    7613 non-null int64
dtypes: int64(4), object(3)
memory usage: 416.5+ KB


In [19]:
train["hashtag"].value_counts()

0    5870
1    1743
Name: hashtag, dtype: int64

In [ ]:
train["hashtag"].head(-5)

In [24]:
feature_col = ["hashtag", "empty_location"]

In [25]:
X = train[feature_col]
y = train["target"]

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

lr = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [32]:
run_model(nb)

0.5703402096819377

In [33]:
run_model(lr)

0.5666608404309521

In [35]:
run_model(rf)

0.5703402096819377

In [38]:
run_model(xgb)

0.5666608404309521

#### Conclusion: not much better than null model, might help when combined with the text

## CREATE A PANDAS SERIES THAT GIVES US THE LEN OF THE TEXT, COMBINE WITH HASHTAG AND MODEL

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [20]:
import re

def text_len(df):
    return len(df.split(" "))

def hashtag(df):
    if re.search(r'#\w+', df):
        return 1
    else:
        return 0
    

    
def run_model(model):
    return cross_val_score(model, X, y, cv = 5, scoring = "accuracy").mean()

In [8]:
##s = "i am a boy"

In [16]:
###len(s.split(" "))

In [18]:
train["length"] = train["text"].apply(text_len)
train["hashtag"] = train["text"].apply(hashtag)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
length      7613 non-null int64
hashtag     7613 non-null int64
dtypes: int64(4), object(3)
memory usage: 416.5+ KB


In [21]:
feature_cols = ["hashtag", "length"]

X = train[feature_cols]
y = train["target"]

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

lr = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [23]:
run_model(lr)

0.5713900777689634

In [24]:
run_model(nb)

0.5703402096819377

In [25]:
run_model(rf)

0.5623266839242815

In [27]:
run_model(xgb)

0.5667921607607251

#### Conclusion: Not much better than NULL MODEL

## COMBINE THE FEATURE ENGINEERED PANDA SERIES WITH TEXT PANDAS SERIES USING FunctionTransformer, make_union and make_pipeline

In [173]:
def text_len(df):
    return len(df.split(" "))

def hashtag(df):
    if re.search(r'#\w+', df):
        return 1
    else:
        return 0
    
def empty_keyword(df):
    if df == "None":
        return 0
    else:
        return 1

def feature_eng (df):
    df["hashtag"] = df["text"].apply(hashtag)
    df["text_len"] = df["text"].apply(text_len)
    df["keyword"] = df["keyword"].fillna("None")
    df["empty_keyword"] = df["keyword"].apply(empty_keyword)
    return df

In [200]:
train_fe = feature_eng(pd.read_csv("../data/train.csv"))
test_fe = feature_eng(pd.read_csv("../data/test.csv"))

In [175]:
train_fe.head()

,id,keyword,location,text,target,hashtag,text_len,empty_keyword
0,1,None,NaN,Our Deeds are the Reason of this #earthquake M...,1,1,13,0
1,4,None,NaN,Forest fire near La Ronge Sask. Canada,1,0,7,0
2,5,None,NaN,All residents asked to 'shelter in place' are ...,1,0,22,0
3,6,None,NaN,"13,000 people receive #wildfires evacuation or...",1,1,9,0
4,7,None,NaN,Just got sent this photo from Ruby #Alaska as ...,1,1,17,0


In [187]:
def get_feature(df):
    return df.loc[:, ['hashtag','text_len']]

In [188]:
from sklearn.preprocessing import FunctionTransformer

# create a stateless transformer from the get_manual function
get_feature_ft = FunctionTransformer(get_feature, validate=False)
type(get_feature_ft)

sklearn.preprocessing._function_transformer.FunctionTransformer

In [189]:
get_feature_ft.transform(train_fe).head()

,hashtag,text_len
0,1,13
1,0,7
2,0,22
3,1,9
4,1,17


In [121]:
def get_text(df):
    return df["text"]

In [122]:
get_text_ft = FunctionTransformer(get_text, validate = False)
type(get_text_ft)

sklearn.preprocessing._function_transformer.FunctionTransformer

In [123]:
get_text_ft.transform(train_fe).shape

(7613,)

In [125]:
union = make_union(tfidf)
X_dtm = union.fit_transform(train_fe["text"])
X_dtm

<7613x21637 sparse matrix of type '<class 'numpy.float64'>'
	with 111497 stored elements in Compressed Sparse Row format>

In [190]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf = TfidfVectorizer(binary = True)

# properly combine the transformers into a FeatureUnion
union = make_union(make_pipeline(get_text_ft, tfidf), get_feature_ft)
X_union = union.fit_transform(train_fe)
X_union.shape


(7613, 21639)

In [202]:
test_union = union.fit_transform(test_fe)
test_union.shape

(3263, 12281)

In [210]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

lr = LogisticRegression()

nb = MultinomialNB()
bnb = BernoulliNB(alpha = 1.2)
rf = RandomForestClassifier()
rf_tune = RandomForestClassifier(n_jobs = -1, class_weight = "balanced_subsample", n_estimators = 200, max_features = "log2")
xgb = XGBClassifier()

def run_model(model):
    return cross_val_score(model, train_fe, y, cv = 5,scoring = "accuracy").mean()

In [211]:
pipe = make_pipeline(union, lr)
run_model(pipe)

0.7326943502303273

In [212]:
pipe = make_pipeline(union, nb)
run_model(pipe)

0.6999924397642128

In [213]:
pipe = make_pipeline(union, bnb)
run_model(pipe)

0.7508296756677304

In [214]:
pipe = make_pipeline(union, rf)
run_model(pipe)

0.6884283649729241

In [215]:
pipe = make_pipeline(union, rf_tune)
run_model(pipe)

0.7262622660548393

In [216]:
pipe = make_pipeline(union, xgb)
run_model(pipe)

0.6869820363503075

In [219]:
tfidf = TfidfVectorizer(binary = True)

# properly combine the transformers into a FeatureUnion
union = make_union(make_pipeline(get_text_ft, tfidf), get_feature_ft)

def output_csv_v2(model, file_name):
    pipe = make_pipeline(union, model)
    pipe.fit(train_fe, y)
    pred = pipe.predict(test_fe)
    return pd.DataFrame({"id": test["id"], "target": pred}).to_csv("../submission-file/"+file_name, index = False)

In [220]:
output_csv_v2(lr, "logisstic_regression_make_union.csv")

In [221]:
output_csv_v2(bnb, "bernoulli_nb_make_union.csv")

In [222]:
output_csv_v2(rf_tune, "random_forest_make_union.csv")

In [ ]:
# FINAL COMCLUSION: IT WAS A NICE RUN AND MY BEST MODEL SCORED 0.79959 ON THE KAGGLE LEADERBOARD
## TAKE AWAYS: 
- MY MODEL (0.79959) OUTPERFORMED THE NULL MODEL (0.57055) WITH A REASONABLE MARGIN
- THIS REPOSITORY GIVES A BEGINNER GUIDELINE TO WORKING WITH TEXT DATA
- THE CONCEPT OF FEATURE ENGINEERING WAS APPLIED IN SIMPLE CODES
- CountVectorizer, TfidfVectorizer, Pipeline and FeatureUnion were all touched